In [1]:
from __future__ import print_function

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.layers import Dense, Input, Flatten
from keras.models import Sequential, Model
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image

%matplotlib inline

Using TensorFlow backend.


In [2]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()
K.set_session(sess)

In [3]:
import h5py

f = h5py.File("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5", "r")
print(f.keys())

[u'block1_conv1', u'block1_conv2', u'block1_pool', u'block2_conv1', u'block2_conv2', u'block2_pool', u'block3_conv1', u'block3_conv2', u'block3_conv3', u'block3_conv4', u'block3_pool', u'block4_conv1', u'block4_conv2', u'block4_conv3', u'block4_conv4', u'block4_pool', u'block5_conv1', u'block5_conv2', u'block5_conv3', u'block5_conv4', u'block5_pool', u'input_2']


In [4]:
class CONFIG:
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    COLOR_CHANNELS = 3
    CONTENT_WEIGHT = 0.025
    STYLE_WEIGHT = 5.0
    TOTAL_VARIATION_WEIGHT = 1.0
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [5]:
def reshape_and_normalize_image(image_path):
    """
    Reshape and normalize the input image (content or style)
    """
    
    image = Image.open(image_path)
    image = image.resize((CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT))
    
    image_array = np.asarray(image, dtype='float32')
    image = np.expand_dims(image_array, axis=0)
    
    # Substract the mean to match the expected input of VGG16
    image = image - CONFIG.MEANS
    
    return image

In [6]:
# Get content_image from file
content_image_path = "./images/monet.jpg"
content_image = reshape_and_normalize_image(content_image_path)

In [7]:
style_image_path = "./images/louvre_small.jpg"
style_image = reshape_and_normalize_image(style_image_path)

In [8]:
# Generate a random noise_image
generated_image = np.random.uniform(-20, 20, (1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.COLOR_CHANNELS)) \
                    .astype('float32')

# Set the input_image to be a weighted average of the content_image and a noise_image
generated_image = generated_image * 0.6 + content_image * (1 - 0.6)

In [9]:
input_tensor = tf.Variable(np.zeros((1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.COLOR_CHANNELS)),
                           dtype='float32', name="input_tensor")

In [10]:
model = vgg19.VGG19(weights=None, input_tensor=input_tensor, include_top=False)
model.load_weights("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [16]:
# flatten = Flatten(input_shape=(None, 300, 400, 3))(model.output)
# dense_outputs = Dense(2, activation='softmax')(flatten)

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [51]:
#Create your own model 
# prediction_model = Model(inputs=model.input, outputs=model.output)

sess.run(input_tensor.assign(content_image))

print(sess.run(model.get_layer("input_1").output))
print(content_image)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
# prediction_model.summary()

[[[[  41.31999969   53.22100067   70.06099701]
   [  44.31999969   54.22100067   74.06099701]
   [  46.31999969   52.22100067   73.06099701]
   ..., 
   [ 100.31999969   73.22100067   74.06099701]
   [ 102.31999969   76.22100067   74.06099701]
   [ 104.31999969   78.22100067   76.06099701]]

  [[  39.31999969   49.22100067   67.06099701]
   [  46.31999969   56.22100067   76.06099701]
   [  52.31999969   58.22100067   79.06099701]
   ..., 
   [ 104.31999969   75.22100067   76.06099701]
   [ 103.31999969   77.22100067   75.06099701]
   [ 102.31999969   76.22100067   74.06099701]]

  [[  39.31999969   49.22100067   67.06099701]
   [  46.31999969   56.22100067   76.06099701]
   [  51.31999969   57.22100067   78.06099701]
   ..., 
   [ 107.31999969   78.22100067   79.06099701]
   [ 107.31999969   78.22100067   77.06099701]
   [ 104.31999969   75.22100067   74.06099701]]

  ..., 
  [[-112.68000031 -102.77899933 -100.93900299]
   [-111.68000031 -101.77899933  -99.93900299]
   [-113.68000031 -

In [49]:
# prediction_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
# data = np.random.random((1000, 224, 224, 3))
# labels = np.random.randint(2, size=(1000, 2))
# prediction_model.fit(data, labels, epochs=10, batch_size=32)